In [1]:
import numpy as np
from numba import njit, prange
from functions import is_target_blocked, v_of_fy

In [2]:
FY1_start_pos = np.array([17800, 0, 0])
FY1_height = np.array([0, 0, 1800])
FY2_start_pos = np.array([12000, 1400, 0])
FY2_height = np.array([0, 0, 1400])
FY3_start_pos = np.array([6000, -3000, 0])
FY3_height = np.array([0, 0, 700])

M1_start_pos = np.array([20000, 0, 2000])

delta_t = 0.01  # time step(s)
Vm = 300 * -M1_start_pos / np.linalg.norm(M1_start_pos)  # speed of missiles(m/s)

g = np.array([0, 0, -9.8])  # the gravity acceleration
V_cloud = np.array([0, 0, -3])  # the velocity of the cloud(m/s)

In [3]:
from scipy.optimize import differential_evolution
@njit(fastmath=True, parallel=True, cache=True, nogil=True)
def objective(x):
    theta1, speed1, t_111, t_112, theta2, speed2, t_211, t_212, theta3, speed3, t_311, t_312 = x
    V1 = v_of_fy(speed1, theta1)
    V2 = v_of_fy(speed2, theta2)
    V3 = v_of_fy(speed3, theta3)
    T1 = np.sqrt(np.sum(M1_start_pos**2)) / 300
    
    steps = int(T1 / delta_t)
    cloud_center_1 = FY1_start_pos + FY1_height + V1 * (t_111 + t_112) + 0.5 * g * t_112**2
    cloud_center_2 = FY2_start_pos + FY2_height + V2 * (t_211 + t_212) + 0.5 * g * t_212**2
    cloud_center_3 = FY3_start_pos + FY3_height + V3 * (t_311 + t_312) + 0.5 * g * t_312**2
    counter = 0
    for i in prange(steps):
        t = i * delta_t
        M1_pos_now = M1_start_pos + Vm * t
        cloud_center_1_now = cloud_center_1 + V_cloud * (t - t_111 - t_112)
        cloud_center_2_now = cloud_center_2 + V_cloud * (t - t_211 - t_212)
        cloud_center_3_now = cloud_center_3 + V_cloud * (t - t_311 - t_312)
        if (is_target_blocked(M1_pos_now, cloud_center_1_now, t - t_111 - t_112) + is_target_blocked(M1_pos_now, cloud_center_2_now, t - t_211 - t_212) + is_target_blocked(M1_pos_now, cloud_center_3_now, t -  t_311 - t_312)):
            counter += 1
    return -counter * delta_t

In [5]:
bounds = [(0, 2*np.pi), (70, 140), (0, 10), (0, 10), (0, 2*np.pi), (70, 140), (0, 20), (0, 20), (0, 2*np.pi), (70, 140), (0, 20), (0, 20)]
result = differential_evolution(objective, bounds, popsize=32, disp=True, maxiter=3000)
theta1, speed1, t_111, t_112, theta2, speed2, t_211, t_212, theta3, speed3, t_311, t_312 = result.x
max_time = -result.fun
print(f"最长遮蔽时间: {max_time:.2f} s")
print(f"最优参数: theta1={theta1:.3f}, speed1={speed1:.3f}, t_111={t_111:.3f}, t_112={t_112:.3f}, theta2={theta2:.3f}, speed2={speed2:.3f}, t_211={t_211:.3f}, t_212={t_212:.3f}, theta3={theta3:.3f}, speed3={speed3:.3f}, t_311={t_311:.3f}, t_312={t_312:.3f}")

differential_evolution step 1: f(x)= -4.64
differential_evolution step 2: f(x)= -4.64
differential_evolution step 3: f(x)= -4.64
differential_evolution step 4: f(x)= -4.64
differential_evolution step 5: f(x)= -4.64
differential_evolution step 6: f(x)= -4.64
differential_evolution step 7: f(x)= -4.64
differential_evolution step 8: f(x)= -4.64
differential_evolution step 9: f(x)= -4.64
differential_evolution step 10: f(x)= -4.64
differential_evolution step 11: f(x)= -4.64
differential_evolution step 12: f(x)= -4.64
differential_evolution step 13: f(x)= -4.64
differential_evolution step 14: f(x)= -4.64
differential_evolution step 15: f(x)= -4.64
differential_evolution step 16: f(x)= -4.64
differential_evolution step 17: f(x)= -4.64
differential_evolution step 18: f(x)= -4.64
differential_evolution step 19: f(x)= -4.64
differential_evolution step 20: f(x)= -4.64
differential_evolution step 21: f(x)= -4.64
differential_evolution step 22: f(x)= -4.64
differential_evolution step 23: f(x)= -4.

最长遮蔽时间: 8.56 s
最优参数: theta1=0.09, speed1=105.24, t_111=0.65, t_112=0.48, theta2=5.19, speed2=132.24, t_211=6.56, t_212=5.01, theta3=3.42, speed3=114.00, t_311=6.46, t_312=7.95

最长遮蔽时间: 8.58 s
最优参数: theta1=0.09, speed1=109.05, t_111=0.78, t_112=0.37, theta2=5.36, speed2=134.92, t_211=8.47, t_212=4.26, theta3=5.77, speed3=94.83, t_311=8.13, t_312=2.03

最长遮蔽时间: 8.58 s
最优参数: theta1=0.09, speed1=105.34, t_111=0.70, t_112=0.45, theta2=5.46, speed2=137.17, t_211=9.92, t_212=3.69, theta3=2.60, speed3=94.95, t_311=15.72, t_312=16.13

In [ ]:
def calc_single_cloud_time(FY_start_pos, FY_height, V, M_start_pos, t_0, t_1):
    cloud_center = FY_start_pos + FY_height + V * (t_0 + t_1) + 0.5 * g * t_1**2
    M_end_pos = M_start_pos + Vm * (t_0 + t_1)
    T1 = np.linalg.norm(M_end_pos) / 300.0
    steps = int(T1 / delta_t)
    counter = 0
    for i in range(steps):
        t = i * delta_t
        M1_pos_now = M_end_pos + Vm * t
        cloud_center_now = cloud_center + V_cloud * t 
        if is_target_blocked(M1_pos_now, cloud_center_now, t):
            counter += 1
    return counter * delta_t

# 计算每架无人机的有效遮蔽时长
V1 = v_of_fy(speed1, theta1)
V2 = v_of_fy(speed2, theta2)
V3 = v_of_fy(speed3, theta3)

time1 = calc_single_cloud_time(FY1_start_pos, FY1_height, V1, M1_start_pos, t_111, t_112)
time2 = calc_single_cloud_time(FY2_start_pos, FY2_height, V2, M1_start_pos, t_211, t_212)
time3 = calc_single_cloud_time(FY3_start_pos, FY3_height, V3, M1_start_pos, t_311, t_312)

print(f"无人机1烟雾弹有效遮蔽时长: {time1:.2f} s")
print(f"无人机2烟雾弹有效遮蔽时长: {time2:.2f} s")
print(f"无人机3烟雾弹有效遮蔽时长: {time3:.2f} s")

无人机1烟雾弹有效遮蔽时长: 4.59 s
无人机2烟雾弹有效遮蔽时长: 3.97 s
无人机3烟雾弹有效遮蔽时长: 0.00 s


In [ ]:
from functions import fy_throw_position

# 第一颗烟雾弹
throw1_pos = fy_throw_position(FY1_start_pos, FY1_height, theta1, speed1, t_111)
# 第二颗烟雾弹
throw2_pos = fy_throw_position(FY2_start_pos, FY1_height, theta2, speed2, t_211)
# 第三颗烟雾弹
throw3_pos = fy_throw_position(FY3_start_pos, FY1_height, theta3, speed3, t_311)

print(f"第一颗烟雾弹投放点: {throw1_pos}")
print(f"第二颗烟雾弹投放点: {throw2_pos}")
print(f"第三颗烟雾弹投放点: {throw3_pos}")

第一颗烟雾弹起爆点: [1.78685459e+04 6.35583076e+00 1.80000000e+03]
第二颗烟雾弹起爆点: [12398.08842124   629.17032825  1800.        ]
第三颗烟雾弹起爆点: [ 5293.28348023 -3205.36519682  1800.        ]


In [ ]:
from functions import fy_bomb_position
# 第一颗烟雾弹
bomb1_pos = fy_bomb_position(FY1_start_pos, FY1_height, theta1, speed1, t_111, t_112)
# 第二颗烟雾弹
bomb2_pos = fy_bomb_position(FY2_start_pos, FY2_height, theta2, speed2, t_211, t_212)
# 第三颗烟雾弹
bomb3_pos = fy_bomb_position(FY3_start_pos, FY3_height, theta3, speed3, t_311, t_312)

print(f"第一颗烟雾弹起爆点: {bomb1_pos}")
print(f"第二颗烟雾弹起爆点: {bomb2_pos}")
print(f"第三颗烟雾弹起爆点: {bomb3_pos}")

第一颗烟雾弹起爆点: [1.79187708e+04 1.10128712e+01 1.79887442e+03]
第二颗烟雾弹起爆点: [12702.12386492    40.45805041  1276.9779707 ]
第三颗烟雾弹起爆点: [ 4423.0859597  -3458.2364402    390.37176982]
